Let $S(k)$ be the sum of three or more distinct positive integers having the following properties:


No value exceeds k.
The values form a geometric progression.
The sum is maximal.

$S(4) = 4 + 2 + 1 = 7$

$S(10) = 9 + 6 + 4 = 19$

$S(12) = 12 + 6 + 3 = 21$

$S(1000) = 1000 + 900 + 810 + 729 = 3439$

Let $T(n)= \sum_{k=4}^{n}(−1)^{k}S(k)$.

$T(1000) = 2268$

Find $T(10^{17})$.

### Facts

$S(k) = n(1 + r + r^2)$

For a geometric series $(nr^2, nr, n) = (a, b, c)$ we have that $b^2 = ac$

### Ideas
##### 3 case 
Find the prime decomposition of $k = p_1^{\gamma_1} \cdot ... \cdot p_l^{\gamma_l}$ and find the smallest square we can take out, let $k'$ be $k$ divided by this smallest factor. This insures that $k \cdot k'$ is still square. This is our first best guess at $c$ (since we know $ac$ is definately square by this procedure). Then find the factor $f = \frac{k}{k'}$, we know that $fk' \leq k$. so we can find the largest square number $\hat{f} \leq f$ and we have that (obviously) $\hat{f}k' \leq k'$ and since $\hat{f}$ is square $\hat{f}k'k$ is also still square and $b = \sqrt{\hat{f}k'k}$

##### highest $r$ for $ar^2, ar, a$
Given the prime decomposition for a number $k = p_1^{f_1}\cdot... \cdot p_n^{f_n}$ the largest candidate for $r$ is $p_1^{\lfloor \frac{f_1}{2}\rfloor} \cdot ... \cdot p_n^{\lfloor \frac{f_n}{2}\rfloor}$


In [2]:
import numpy as np
import primefac as pf
from __future__ import division
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

/vol/bitbucket/ml3613/gps/gpenv/local/lib/python2.7/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime


## Intial attempt using climbing s calcualtions from 4 up to k

In [11]:
# Given a number in factorint form, turn to int
def multiply(factors):
    prod = 1
    for key in factors:
        prod *= key ** factors[key]
    return int(prod)

# Return the primes with floored(power/2)
def get_highest_two(factors):
    highest = {}
    for key in factors:
        if factors[key] >= 2:
            highest[key] = factors[key]
    for key in highest:
        highest[key] = (highest[key] - (highest[key] % 2)) / 2
    return highest

# Given the numerator of r, get the largest candidate sequence
def to_seq(k, numerator):
    denominator = numerator - 1
    seq = [k]
    elem = k
    while (elem * denominator) % numerator == 0:
        elem = (elem * denominator) / numerator
        seq.append(int(elem))
    return seq

# This returns the numerator and denominator or the r factor
def get_r(k):
    factors = pf.factorint(k)
    numerator = multiply(get_highest_two(factors))
    return numerator, numerator - 1

# Main work horse
def climb_s(k, base_seq):
    if k == 4:
        return [4, 2, 1]
    numerator, denominator = get_r(k)
    
    if numerator > 1: # we have a sequence where k is at least ar^{3}
        seq = to_seq(k, numerator)
        if np.sum(base_seq) > np.sum(seq):
            return base_seq
        else:
            return seq
    else:
        return base_seq

# Just for testing
def s(k, verbose=False):
    seq = [0]
    previous_seq = [0]
    for i in range(4, k + 1):
        previous_seq = seq
        seq = climb_s(i, seq)
        if verbose:
            if seq != previous_seq:
                print i, ':', pf.factorint(i), ':', seq
    return seq

def t(k):
    total = 0
    seq = [0]
    for i in range(4, k + 1):
        seq = climb_s(i, seq)
        total += ((-1)**i) * np.sum(seq)
    return total

time: 34.6 ms


In [24]:
t(10**6)

1869227

time: 29.3 s


In [9]:
%prun t(10**5)

 time: 3.75 s


## Testing optimisations

In [125]:
def is_candidate(p):
    factors = pf.factorint(p)
    numerator = multiply(get_highest_two(factors))
    if numerator > 1:
        return True
    else:
        return False

def check_list(lower, upper):
    if upper - lower <= 10:
        return [num for num in np.arange(lower, upper) if is_candidate(num)]
    #candidates = [num for num in np.arange(k) if is_candidate(num)]
    else:
        pivot = int(np.round((upper - lower)/2.0))
        candidates = np.concatenate([check_list(lower, lower + pivot),
                                     check_list(lower + pivot, upper)],
                                    axis = 0)
    return candidates

def efficient_t(k):
    primes = pf.primes(np.sqrt(k))
    

time: 6.66 ms


In [88]:
primes = pf.primes(np.sqrt(101))

time: 1.63 ms


In [133]:
k = 10 ** 3
list_to_check = check_list(0, k)
print len(list_to_check)
print len(list_to_check)/k

391
0.391
time: 27.2 ms
